In [370]:
#Pruebas de predicciones con nuevo registro para clasificar 1
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from imblearn.over_sampling import SMOTE

# Entrenamiento previo del modelo

columnas_relevantes = ['SubRubroCodigo', 'RubroCodigo', 'SubclasificacionCodigo', 'ClasificacionCodigo']
data_relevante = data[columnas_relevantes]

# Escalado
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_relevante)

# Aplicación de PCA
pca = PCA(n_components=2)
data_pca = pca.fit_transform(data_scaled)
data['PC1'] = data_pca[:, 0]
data['PC2'] = data_pca[:, 1]

# Creación de la variable objetivo
encoder = LabelEncoder()
data['Target10'] = encoder.fit_transform(data['RedesNeuronales'])

# División en entrenamiento y prueba
X = data[['PC1', 'PC2']]
y = data['Target10']

# Aplicar SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Entrenamiento del modelo
rf_model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
rf_model.fit(X_resampled, y_resampled)

# Definición del modelo de redes neuronales
nn_model = Sequential()
nn_model.add(Input(shape=(X_resampled.shape[1],)))
nn_model.add(Dense(128, activation='relu'))
nn_model.add(Dropout(0.4))
nn_model.add(Dense(64, activation='relu'))
nn_model.add(Dropout(0.3))
nn_model.add(Dense(32, activation='relu'))
nn_model.add(Dense(len(np.unique(y_resampled)), activation='softmax'))
optimizer = Adam(learning_rate=0.001)
nn_model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
nn_model.fit(X_resampled, y_resampled, epochs=70, batch_size=64, verbose=1)

# Clasificación de un nuevo producto
nuevo_producto = pd.DataFrame([{
    'Nombre': 'preueba3',
    'RubroCodigo': 26,
    'SubRubroCodigo': 50,
    'ClasificacionCodigo': 2,
    'SubclasificacionCodigo': 5    
}])

# Escalado del nuevo producto
nuevo_producto_scaled = scaler.transform(nuevo_producto[columnas_relevantes])

# Transformación PCA
nuevo_producto_pca = pca.transform(nuevo_producto_scaled)

# Predicción con Random Forest
rf_pred = rf_model.predict(nuevo_producto_pca)
rf_clase_predicha = encoder.inverse_transform(rf_pred)
print(f"Predicción con Random Forest: {rf_clase_predicha[0]}")

# Predicción con Redes Neuronales
nn_pred_prob = nn_model.predict(nuevo_producto_pca)
nn_clase_predicha = encoder.inverse_transform([np.argmax(nn_pred_prob)])
print(f"Predicción con Redes Neuronales: {nn_clase_predicha[0]}")

# Resultados combinados
nuevo_producto['PC1'] = nuevo_producto_pca[0, 0]
nuevo_producto['PC2'] = nuevo_producto_pca[0, 1]
nuevo_producto['Prediccion_RF'] = rf_clase_predicha[0]
nuevo_producto['Prediccion_NN'] = nn_clase_predicha[0]
print("Nuevo producto clasificado:")
print(nuevo_producto)


Epoch 1/70
113/113 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6077 - loss: 0.6572
Epoch 2/70
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6621 - loss: 0.6157
Epoch 3/70
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6716 - loss: 0.5983
Epoch 4/70
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6785 - loss: 0.5813
Epoch 5/70
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6706 - loss: 0.5816
Epoch 6/70
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6799 - loss: 0.5739
Epoch 7/70
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6788 - loss: 0.5687
Epoch 8/70
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6666 - loss: 0.5792
Epoch 9/70
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6774 - loss: 0.5658
Epoch 10/70
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6731 - loss: 0.5718
Epoch 11/70
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6766 - loss: 0.5649
Epoch 12/70
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8243 - loss: 0.3792
Epoch 69/70
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8199 - loss: 0.3789
Epoch 70/70
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8265 - loss: 0.3736
Predicción con Random Forest: Anómalo


C:\Users\Usuario\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step
Predicción con Redes Neuronales: Anómalo
Nuevo producto clasificado:
     Nombre  RubroCodigo  SubRubroCodigo  ClasificacionCodigo  \
0  preueba3           26              50                    2   

   SubclasificacionCodigo       PC1       PC2 Prediccion_RF Prediccion_NN  
0                       5 -1.792914  0.402075       Anómalo       Anómalo  


In [371]:
#Pruebas de predicciones con nuevo registro para clasificar 2
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# Simulación del dataset original
data = pd.DataFrame({
    'RubroCodigo': np.random.randint(1, 10, 100),
    'SubRubroCodigo': np.random.randint(1, 20, 100),
    'ClasificacionCodigo': np.random.randint(1, 15, 100),
    'SubclasificacionCodigo': np.random.randint(1, 10, 100),
    'Stock': np.random.randint(50, 500, 100),
    'cluster_dbscan': np.random.choice([-1, 0], 100),
    'Outlier': np.random.choice([0, 1], 100)
})

# Crear la columna target (clasificación)
def clasificar_producto(row):
    if row['cluster_dbscan'] == -1 or row['Outlier'] == 1:
        return 'Anómalo'
    else:
        return 'Normal'

data['target12'] = data.apply(clasificar_producto, axis=1)

# Seleccionar columnas relevantes
columnas_relevantes = ['RubroCodigo', 'SubRubroCodigo', 'ClasificacionCodigo', 'SubclasificacionCodigo', 'Stock']
X = data[columnas_relevantes]
y = data['target12']

# Escalar las características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Aplicar PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Codificar la variable objetivo
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_pca, y_categorical, test_size=0.3, random_state=42)

# Definición del modelo
model = Sequential()
model.add(Input(shape=(X_train.shape[1],)))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))

# Compilar el modelo
optimizer = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Entrenar el modelo
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32, verbose=1)

# Clasificar un nuevo producto
nuevo_producto = pd.DataFrame([{
    'RubroCodigo': 3,
    'SubRubroCodigo': 12,
    'ClasificacionCodigo': 5,
    'SubclasificacionCodigo': 8,
    'Stock': 50
}])

# Preprocesamiento del nuevo producto
nuevo_producto_scaled = scaler.transform(nuevo_producto)  # Escalar datos
nuevo_producto_pca = pca.transform(nuevo_producto_scaled)  # Transformar con PCA

# Predicción con el modelo entrenado
prediccion = model.predict(nuevo_producto_pca)
probabilidades = prediccion[0]  # Probabilidades para cada clase

# Interpretar la predicción
clase_predicha = encoder.inverse_transform([np.argmax(prediccion)])
print(f"Clase predicha: {clase_predicha[0]}")
print(f"Probabilidades: {probabilidades}")

# Agregar la predicción al DataFrame del nuevo producto
nuevo_producto['Prediccion'] = clase_predicha[0]
nuevo_producto['Probabilidad_Anómalo'] = probabilidades[1]
nuevo_producto['Probabilidad_Normal'] = probabilidades[0]

print("Nuevo producto clasificado:")
print(nuevo_producto)


Epoch 1/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 285ms/step - accuracy: 0.3985 - loss: 0.7148 - val_accuracy: 0.7000 - val_loss: 0.6763
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.6977 - loss: 0.6658 - val_accuracy: 0.7667 - val_loss: 0.6511
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.8030 - loss: 0.6136 - val_accuracy: 0.7667 - val_loss: 0.6358
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.8290 - loss: 0.5824 - val_accuracy: 0.7667 - val_loss: 0.6247
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.8251 - loss: 0.5557 - val_accuracy: 0.7667 - val_loss: 0.6149
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.8017 - loss: 0.5545 - val_accuracy: 0.7667 - val_loss: 0.6075
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.8134 - loss: 0.5344 - val_accuracy: 0.7667 - val_loss: 0.6026
Epoch 8/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.8017 - loss: 0.5193 - val_accuracy: 0.7667 - val_loss: 0.6011

In [373]:
#Pruebas de predicciones con nuevo registro para clasificar 3
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# Simulación del dataset original
data = pd.DataFrame({
    'RubroCodigo': np.random.randint(1, 10, 100),
    'SubRubroCodigo': np.random.randint(1, 20, 100),
    'ClasificacionCodigo': np.random.randint(1, 15, 100),
    'SubclasificacionCodigo': np.random.randint(1, 10, 100),
    'Stock': np.random.randint(50, 500, 100),
    'cluster_dbscan': np.random.choice([-1, 0], 100),
    'Outlier': np.random.choice([0, 1], 100)
})

# Crear la columna target (clasificación)
def clasificar_producto(row):
    if row['cluster_dbscan'] == -1 or row['Outlier'] == 1:
        return 'Anómalo'
    else:
        return 'Normal'

data['target12'] = data.apply(clasificar_producto, axis=1)

# Seleccionar columnas relevantes
columnas_relevantes = ['RubroCodigo', 'SubRubroCodigo', 'ClasificacionCodigo', 'SubclasificacionCodigo', 'Stock']
X = data[columnas_relevantes]
y = data['target12']

# Escalar las características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Aplicar PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Codificar la variable objetivo
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_pca, y_categorical, test_size=0.3, random_state=42)

# Definición del modelo
model = Sequential()
model.add(Input(shape=(X_train.shape[1],)))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))

# Compilar el modelo
optimizer = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Entrenar el modelo
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32, verbose=1)

# Clasificar un nuevo producto
nuevo_producto = pd.DataFrame([{
    'RubroCodigo': 26,
    'SubRubroCodigo': 50,
    'ClasificacionCodigo': 2,
    'SubclasificacionCodigo': 5,
    'Stock': 50
}])

# Preprocesamiento del nuevo producto
nuevo_producto_scaled = scaler.transform(nuevo_producto)  # Escalar datos
nuevo_producto_pca = pca.transform(nuevo_producto_scaled)  # Transformar con PCA

# Predicción con el modelo entrenado
prediccion = model.predict(nuevo_producto_pca)
probabilidades = prediccion[0]  # Probabilidades para cada clase

# Interpretar la predicción
clase_predicha = encoder.inverse_transform([np.argmax(prediccion)])
print(f"Clase predicha: {clase_predicha[0]}")
print(f"Probabilidades: {probabilidades}")

# Agregar la predicción al DataFrame del nuevo producto
nuevo_producto['Prediccion'] = clase_predicha[0]
nuevo_producto['Probabilidad_Anómalo'] = probabilidades[1]
nuevo_producto['Probabilidad_Normal'] = probabilidades[0]

print("Nuevo producto clasificado:")
print(nuevo_producto)

#da buen resultado !!!!!!!!!

Epoch 1/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 239ms/step - accuracy: 0.3901 - loss: 0.7233 - val_accuracy: 0.7333 - val_loss: 0.6664
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.7165 - loss: 0.6456 - val_accuracy: 0.7333 - val_loss: 0.6404
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.8433 - loss: 0.5859 - val_accuracy: 0.7333 - val_loss: 0.6262
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.8199 - loss: 0.5620 - val_accuracy: 0.7333 - val_loss: 0.6209
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.8433 - loss: 0.5008 - val_accuracy: 0.7333 - val_loss: 0.6229
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.8238 - loss: 0.5140 - val_accuracy: 0.7333 - val_loss: 0.6329
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.8316 - loss: 0.5001 - val_accuracy: 0.7333 - val_loss: 0.6515
Epoch 8/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.8472 - loss: 0.4705 - val_accuracy: 0.7333 - val_loss: 0.6758